In [ ]:
# General setup for all examples
import foolbox.ext.native as fbn
import torchvision.models as models

# instantiate a model
model = models.resnet18(pretrained=True).eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = fbn.models.PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

# get data and test the model
images, labels = fbn.utils.samples(fmodel, dataset='imagenet', batchsize=4)
print(fbn.utils.accuracy(fmodel, images, labels))
# -> 1.0

### Minimum example to use Brendel & Bethge attack

In [ ]:
# apply the Brendel & Bethge attack
attack = fbn.attacks.L2BrendelBethgeAttack(fmodel)
adversarials = attack(
    images,
    labels,
    lr=1,
    tensorboard=False,
)
print(fbn.utils.accuracy(fmodel, adversarials, labels))
# -> 0.0

### Using Brendel & Bethge with starting points selected from data set (to ensure valid starting points)

In [ ]:
# get all data
images, labels = fbn.utils.samples(fmodel, dataset='imagenet', batchsize=20)

# split images and labels into 'batches' (just to mirror the batches of standard large-scale datasets)
batches = [(images[:10], labels[:10]), (images[10:], labels[10:])]

# create attack that picks adversarials from given dataset of samples
init_attack = fbn.attacks.PrecomputedSamplesAttack(fmodel)

init_attack.feed(batches[0][0])   # feed 1st batch of inputs
init_attack.feed(batches[1][0])   # feed 2nd batch of inputs

# apply the Brendel & Bethge attack
attack = fbn.attacks.L2BrendelBethgeAttack(fmodel)
adversarials = attack(
    images,
    labels,
    lr=1,
    init_attack=init_attack
)
print(fbn.utils.accuracy(fmodel, adversarials, labels))
# -> 0.0